Model Training


In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from xgboost import XGBRegressor


In [33]:
df=pd.read_csv('data/stud.csv')

In [34]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [35]:
#Predict Math score in this proj
X=df.drop(columns=['math_score'],axis=1)

In [36]:
y=df['math_score']

In [37]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [38]:
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

Apply Column Transformer 

In [39]:
#Perform Encoding and standardization
num_features=X.select_dtypes(exclude="object").columns
cat_features=X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer=StandardScaler()
oh_transformer=OneHotEncoder()

preprocessor=ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,cat_features),
        ("StandardScaler",numeric_transformer,num_features)
    ]
)

In [40]:
X.shape

(1000, 7)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=42)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


Create an evaluation metrics function

In [42]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2_square = r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [46]:
models = {
    'Linear Regression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'KNN Regressor': KNeighborsRegressor(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost Regressor': XGBRegressor(),
    'Ada boost': AdaBoostRegressor()
}

model_list = []
r2_list = []

for model_name, model in models.items():

    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae, model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    model_list.append(model_name)
    print(model_name)
    print("Training:")
    print("MAE", model_train_mae)
    print("MSE", model_train_mse)
    print("RMSE", model_train_rmse)
    print("R2", model_train_r2)
    
    print("Testing:")
    print("MAE", model_test_mae)
    print("MSE", model_test_mse)
    print("RMSE", model_test_rmse)
    print("R2", model_test_r2)
    r2_list.append(model_test_r2)
    print("---------------------------------------")

Linear Regression
Training:
MAE 4.267761840820312
MSE 28.349118786007168
RMSE 5.324389052840445
R2 0.874254003462845
Testing:
MAE 4.217325439453125
MSE 29.124002436101435
RMSE 5.3966658629288355
R2 0.8803148108309192
---------------------------------------
Lasso
Training:
MAE 5.205261564799369
MSE 43.46111315450666
RMSE 6.592504315850439
R2 0.8072228972801443
Testing:
MAE 5.155719544015369
MSE 42.47580702071846
RMSE 6.517346624257334
R2 0.825445523514914
---------------------------------------
Ridge
Training:
MAE 4.265005112727168
MSE 28.337741791088874
RMSE 5.323320560617111
R2 0.8743044675204547
Testing:
MAE 4.211112826071164
MSE 29.056601626694825
RMSE 5.390417574427312
R2 0.8805917946912825
---------------------------------------
KNN Regressor
Training:
MAE 4.50425
MSE 32.46085
RMSE 5.697442408660223
R2 0.8560159149035753
Testing:
MAE 5.587999999999999
MSE 52.088
RMSE 7.217201673779111
R2 0.7859441830800236
---------------------------------------
Decision Tree
Training:
MAE 0.01875

In [47]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['Model name','R2 Score']).sort_values(by=["R2 Score"],ascending=False)

,Model name,R2 Score
2,Ridge,0.880592
0,Linear Regression,0.880315
5,Random Forest,0.853355
7,Ada boost,0.848985
6,XGBoost Regressor,0.827797
1,Lasso,0.825446
3,KNN Regressor,0.785944
4,Decision Tree,0.747553
